<a href="https://colab.research.google.com/github/SDS-AAU/SDS-master/blob/master/M3/exercises/CNN_workshop_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
install.packages('reticulate')
install.packages('magrittr')

In [ ]:
install.packages("keras") 


In [ ]:
library(tidyverse)
library(magrittr)
library(keras)

In [ ]:
# Let's start by downloading and exploring the data
temp <- tempfile()
download.file('https://storage.googleapis.com/sds-file-transfer/hot-dog-not-hot-dog.zip',temp)
unzip(temp)
unlink(temp)

In [ ]:
list.files(path = "test") %>% head()
list.files(path = "train") %>% head()

# Preprocessing

Same as in the former notebook

In [ ]:
# how much data?
train_size <- c(list.files(path = "train/hot_dog"), list.files(path = "train/not_hot_dog")) %>% length()
test_size <- c(list.files(path = "test/hot_dog"), list.files(path = "test/not_hot_dog")) %>% length()

In [ ]:
class_list <- c('hot_dog', 'not_hot_dog')
output_n <- length(class_list)
img_width <- 224
img_height <- 224
target_size <- c(img_width, img_height)
channels <- 3
train_files_path <- 'train'
test_files_path <- 'test'
batch_size <- 32

In [ ]:
train_data_gen = image_data_generator(rescale = 1/255, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = TRUE)
test_data_gen <- image_data_generator(rescale = 1/255)  

In [ ]:
train_image_array_gen <- flow_images_from_directory(train_files_path, train_data_gen, target_size = target_size, class_mode = "binary", 
                                                    classes = class_list,batch_size = batch_size, seed = 1337)

test_image_array_gen <- flow_images_from_directory(test_files_path, test_data_gen, target_size = target_size, class_mode = "binary",
                                          classes = class_list, batch_size = batch_size, seed = 1337)

# Loading pretrained model

Keras has some great built-in applications - basically pretrained models with some nice functional overhead. We will start by loadeing VGG16 (a rather large model with many many many layers that has been trained on imagenet)



In [ ]:
# create the base pre-trained model
base_model <- application_vgg16(
  include_top = FALSE,
  weights='imagenet',
  input_shape = c(img_height, img_width, channels))

In [ ]:
predictions <- base_model$output %>% 
  layer_flatten(trainable = T) %>%
  layer_dense(1024, activation = 'relu', trainable = T) %>%
  layer_dropout(0.4, trainable = T) %>%
  layer_dense(84, activation = 'relu', trainable=T) %>%  
  layer_dense(units = 1, activation = 'sigmoid', trainable=T)

In [ ]:
# this is the model we will train
model <- keras_model(inputs = base_model$input, outputs = predictions)

In [ ]:
# let's visualize layer names and layer indices to see how many layers we should freeze:
model %>% summary()

In [ ]:
# first: train only the top layers 
for (layer in base_model$layers)
  layer$trainable <- FALSE

In [ ]:
# compile the model (should be done *after* setting layers to non-trainable)
# We this time also set a vry low larning rate
model %>% compile(optimizer = optimizer_adam(lr = 0.0001), # Notice this
                  loss = 'binary_crossentropy',
                  metric = 'accuracy')

In [ ]:
# train the model on the new data for a few epochs
model %>% fit_generator(
  train_image_array_gen,
  steps_per_epoch = as.integer((train_size / batch_size) ),
  epochs = 10, 
  verbose = FALSE,
  callbacks = list(callback_early_stopping(patience = 2))
  )

In [ ]:
hist_train2 %>% plot()

In [ ]:
# we chose to train the top 2 blocks, i.e. we will freeze the first layers and unfreeze the rest:
for (i in 1:(length(layers)-2))
  layers[[i]]$trainable <- FALSE
for (i in (length(layers)-1):length(layers))
  layers[[i]]$trainable <- TRUE

In [ ]:
# we need to recompile the model for these modifications to take effect
model %>% compile(optimizer = 'adam', 
                  loss = 'binary_crossentropy',
                  metric = 'accuracy')

In [ ]:
# we train our model again (this time fine-tuning the top 2  blocks
# alongside the top Dense layers
# train the model on the new data for a few epochs
hist_train2 <- model %>% fit_generator(
  train_image_array_gen,
  steps_per_epoch = as.integer((train_size / batch_size)),
  epochs = 10, 
  validation_data = test_image_array_gen,
  validation_steps = as.integer((test_size / batch_size)),
  verbose = FALSE,
  callbacks = list(callback_early_stopping(patience = 2))
  )

In [ ]:
hist_train2 %>% plot()

In [ ]:
model %>% evaluate_generator(test_image_array_gen, steps=500)

# some own image

![](https://do512family.com/wp-content/uploads/2018/04/18222376_1528395153845930_2934228273602805559_n.jpg)


![](http://myminidoxie.com/wp-content/uploads/2012/11/which-came-first-the-wiener-or-the-dachshund.jpg)


In [ ]:
download.file('https://do512family.com/wp-content/uploads/2018/04/18222376_1528395153845930_2934228273602805559_n.jpg',
'wiener1.jpg', mode = 'wb')

download.file('http://myminidoxie.com/wp-content/uploads/2012/11/which-came-first-the-wiener-or-the-dachshund.jpg',
'wiener2.jpg', mode = 'wb')

In [ ]:
img <- image_load('wiener2.jpg', target_size = target_size) %>% image_to_array() %>% array_reshape(c(1, dim(.)))

In [ ]:
res <- predict(model, img)

In [ ]:
ifelse(res == 0, 'hot_dog', 'not_hot_dog')